In [3]:
import os
import cohere
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the Cohere API key from environment variables
cohere_api_key = os.getenv('COHERE_API_KEY')
if not cohere_api_key:
    raise ValueError("Cohere API key not found. Please set it in the .env file.")

cohere_client = cohere.Client(cohere_api_key)
class LLMInterface:
    def query(self, prompt):
        try:
            response = cohere_client.generate(prompt=prompt)
            return response.generations[0].text
        except Exception as e:
            print(f"Error querying Cohere API: {e}")
            return None

class LangChainModel:
    def __init__(self, llm_interface):
        self.llm_interface = llm_interface

    def get_suggestions(self, parameters):
        prompt = self.create_prompt(parameters)
        return self.llm_interface.query(prompt)

    def create_prompt(self, parameters):
        prompt = "Generate suggestions based on the following parameters:\n"
        for param in parameters:
            prompt += f"{param['name']}: {param['description']}\n"
        return prompt
    
parameters = [
    {"name": "Relevance", "description": "Measures how relevant the suggested application is to current market trends and user needs."},
    {"name": "Profitability", "description": "Estimates the potential revenue that the suggested application could generate."},
    {"name": "Complexity", "description": "Assesses the technical complexity involved in developing the suggested application."},
    {"name": "User Engagement", "description": "Predicts the level of user engagement and retention for the suggested application."},
    {"name": "Development Time", "description": "Estimates the time required to develop the suggested application."}
]

llm_interface = LLMInterface()
langchain_model = LangChainModel(llm_interface)

suggestions = langchain_model.get_suggestions(parameters)
if suggestions:
    print(suggestions)
else:
    print("Failed to get suggestions.")


 Here are some suggestions for a new mobile application, along with assessments regarding their Relevance, Profitability, Complexity, User Engagement, and Development Time:

1. StayFocusApp (High Relevance, Moderate Profitability, Medium Complexity, High User Engagement, Short Development Time)
   - A productivity application that helps users stay focused on their tasks by blocking distracting apps and websites for a predetermined time.
   - High relevance due to the increasing need for efficient productivity tools in a multitasking world.
   - Moderate profitability potential through subscription plans or one-time purchases.
   - Medium complexity in terms of development, requiring basic AI and behavior-tracking features.
   - High user engagement potential due to its value in improving productivity.
   - Short development time, ranging from a few months to a year.

2. EcoGenius (High Relevance, High Profitability, Medium Complexity, High User Engagement, Medium Development Time)
   -

In [2]:
import os
from langchain_cohere import ChatCohere
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
#from dotenv import load_dotenv # Import the load_dotenv function from the dotenv module

#load_dotenv(dotenv_path='../.env')

chat_llm = ChatCohere(
    cohere_api_key=os.getenv("COHERE_API_KEY")
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        ("system", "{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

memory = ChatMessageHistory()

def get_memory(session_id):
    return memory

chat_chain = prompt | chat_llm | StrOutputParser()

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)


current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

while (question := input("> ")) != "exit":

    response = chat_with_message_history.invoke(
        {
            "context": current_weather,
            "question": question,
            
        }, 
        config={
            "configurable": {"session_id": "none"}
        }
    )
    
    print(response)

In [13]:
import os
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime
from sqlalchemy.orm import declarative_base, sessionmaker
from dotenv import load_dotenv
from langchain_cohere import ChatCohere
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.tools import YouTubeSearchTool
from uuid import uuid4
from datetime import datetime

# Load environment variables from .env file
load_dotenv()

# Set up SQLite database
DATABASE_URL = "sqlite:///C:/Users/jarmo/gits/interface_app_once_a_day/backend/chat_history.db"
Base = declarative_base()
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Define ChatMessage model
class ChatMessage(Base):
    __tablename__ = "chat_messages"
    id = Column(Integer, primary_key=True, index=True)
    session_id = Column(String, index=True)
    role = Column(String)
    content = Column(Text)
    timestamp = Column(DateTime, default=datetime.utcnow)

# Create the database tables
Base.metadata.create_all(bind=engine)

# Function to get a new database session
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Function to save chat message to the database
def save_chat_message(session_id, role, content, db):
    db_message = ChatMessage(session_id=session_id, role=role, content=content)
    db.add(db_message)
    db.commit()
    db.refresh(db_message)
    return db_message

# Function to get chat message history from the database
def get_chat_message_history(session_id, db):
    return db.query(ChatMessage).filter(ChatMessage.session_id == session_id).all()

# Custom ChatMessageHistory class
class ChatMessageHistory:
    def __init__(self, messages):
        self.messages = messages

    def add_messages(self, new_messages):
        self.messages.extend(new_messages)

# Initialize Cohere client
llm = ChatCohere(
    cohere_api_key=os.getenv("COHERE_API_KEY")
)

# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You find movies from a genre or plot.",
        ),
        ("human", "{input}"),
    ]
)

# Create the movie chat chain
movie_chat = prompt | llm | StrOutputParser()

# Initialize YouTube search tool
youtube = YouTubeSearchTool()

# Function to call YouTube trailer search
def call_trailer_search(input):
    input = input.replace(",", " ")
    return youtube.run(input)

# Define tools
tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=movie_chat.invoke,
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word trailer. Return a link to a YouTube video.",
        func=call_trailer_search,
    ),
]

# Pull agent prompt from hub
agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Function to get chat message history for a session
def get_memory(session_id):
    db = next(get_db())
    messages = get_chat_message_history(session_id=session_id, db=db)
    return ChatMessageHistory(messages)

# Create chat agent with message history
chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Generate a unique session ID
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

# Main loop to handle user input and generate responses
while (q := input("> ")) != "exit":
    db = next(get_db())
    save_chat_message(SESSION_ID, "human", q, db)
    
    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID, "message_history": get_memory(SESSION_ID)}},
    )
    
    save_chat_message(SESSION_ID, "system", response["output"], db)
    print(response["output"])

c:\Users\jarmo\anaconda3\envs\learn_langchain-env\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Session ID: 50be868f-d4ec-4ac4-a744-dc3c9f5dd805
Hi there! How can I assist you today? I can help with a variety of movie-related queries, whether you're looking for recommendations, trivia, or just want to chat about your favorite films. Feel free to ask away!
Yes, I'm ready to assist you with any movie-related queries you may have. How can I help you today?
I'm sorry, I don't understand your question. Could you rephrase it, please?
Hello, Jarmo! It's a pleasure to meet you. How can I assist you today? I can help with any movie-related queries you might have, whether it's discussing your favourite films, providing recommendations, or searching for specific movie trailers. Just let me know how I can be of service!
I'm sorry, I didn't understand your question. Could you please rephrase or provide more context?
You are a human user who is interacting with Assistant, a large language model trained by OpenAI. You are engaging in a conversation about movies and movie-related topics.
I'm sor